In [69]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession,DataFrame
from pyspark import StorageLevel
from pyspark.sql.functions import col, round, to_date, year, month, sum, avg

conf = (
     SparkConf()
    .setAppName('Simple_Spark')
    .setMaster('local[*]')
)


# Spark сессия
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    # .enableHiveSupport()
    .getOrCreate()
)

In [70]:
spark.sparkContext.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.host', '192.168.100.11'),
 ('spark.app.id', 'local-1726511225150'),
 ('spark.executor.i

cache — это метод в Apache Spark, который используется для кэширования данных в памяти. Он позволяет избежать повторного выполнения вычислений при повторном доступе к данным. Кэширование может существенно повысить производительность Spark-приложений, особенно если одни и те же данные используются многократно в различных этапах вычислений, то есть чаще всего при join операциях.

Основная логика cache следующая - 

> Данные сохраняются в оперативной памяти всех рабочих узлов (executors) кластера.
При повторном доступе к данным Spark извлекает их из памяти, а не выполняет вычисления заново.

> Метод cache помечает DataFrame или RDD для кэширования, но фактически кэширование происходит только при первом вызове действия (action), которое вызывает выполнение вычислений (например, count, collect, show).

> Если данные не помещаются в память, Spark может использовать метод persist с соответствующим уровнем хранения (storage level), чтобы сохранить их на диск 

In [71]:
# создаем df
data = [(i, f"name_{i * 5}") for i in range(1000000)]
df = spark.createDataFrame(data, ["id", "name"])

In [72]:
# Применяем фильтр и кэшируем результат
filtered_df = (
df
.filter(col("id") % 2 == 0)
)

In [74]:
# кэшируем при action
filtered_df.cache()
filtered_df.count()

24/09/16 21:28:37 WARN CacheManager: Asked to cache already cached data.
24/09/16 21:28:37 WARN TaskSetManager: Stage 4 contains a task of very large size (5038 KiB). The maximum recommended task size is 1000 KiB.


500000

In [16]:
filtered_df.show(10)

+---+-------+
| id|   name|
+---+-------+
|  0| name_0|
|  2|name_10|
|  4|name_20|
|  6|name_30|
|  8|name_40|
| 10|name_50|
| 12|name_60|
| 14|name_70|
| 16|name_80|
| 18|name_90|
+---+-------+
only showing top 10 rows



метод __cache__ использует стандартный уровень хранения MEMORY_ONLY

#### persist

    MEMORY_ONLY: Кэширует данные только в памяти. Если данные не помещаются, они не сохраняются на диск.
    MEMORY_AND_DISK: Кэширует данные в памяти и на диске, если данные не помещаются в память.
    DISK_ONLY: Кэширует данные только на диске.
    MEMORY_ONLY_SER: Кэширует данные в памяти в сериализованном виде. Это уменьшает объем памяти, но увеличивает время доступа.
    MEMORY_AND_DISK_SER: Кэширует данные в памяти и на диске в сериализованном виде.
    OFF_HEAP: Кэширует данные в памяти вне кучи (off-heap), что может уменьшить накладные расходы на сборку мусора (Garbage Collection).


In [80]:
filtered_df.persist(StorageLevel.DISK_ONLY)
filtered_df.count()

24/09/16 22:03:29 WARN TaskSetManager: Stage 15 contains a task of very large size (5038 KiB). The maximum recommended task size is 1000 KiB.
24/09/16 22:03:30 WARN TaskSetManager: Stage 16 contains a task of very large size (5038 KiB). The maximum recommended task size is 1000 KiB.


500000

In [63]:
filtered_df.persist(StorageLevel.MEMORY_AND_DISK_2)
filtered_df.count()

24/09/16 21:22:10 WARN TaskSetManager: Stage 48 contains a task of very large size (5038 KiB). The maximum recommended task size is 1000 KiB.
24/09/16 21:22:11 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/09/16 21:22:11 WARN BlockManager: Block rdd_160_1 replicated to only 0 peer(s) instead of 1 peers
24/09/16 21:22:11 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/09/16 21:22:11 WARN BlockManager: Block rdd_160_3 replicated to only 0 peer(s) instead of 1 peers
24/09/16 21:22:11 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/09/16 21:22:11 WARN BlockManager: Block rdd_160_0 replicated to only 0 peer(s) instead of 1 peers
24/09/16 21:22:11 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/09/16 21:22:11 WARN BlockManager: Block rdd_160_2 replicated to only 0 peer(s) instead of 1 peers
24/09/16 21:22:11 WARN TaskSetManager: Stage 49 contains a task of very large 

500000

In [65]:
filtered_df.persist(StorageLevel.DISK_ONLY_2)
filtered_df.count()

24/09/16 21:24:59 WARN TaskSetManager: Stage 52 contains a task of very large size (5038 KiB). The maximum recommended task size is 1000 KiB.
24/09/16 21:25:00 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/09/16 21:25:00 WARN BlockManager: Block rdd_171_1 replicated to only 0 peer(s) instead of 1 peers
24/09/16 21:25:00 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/09/16 21:25:00 WARN BlockManager: Block rdd_171_0 replicated to only 0 peer(s) instead of 1 peers
24/09/16 21:25:00 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/09/16 21:25:00 WARN BlockManager: Block rdd_171_2 replicated to only 0 peer(s) instead of 1 peers
24/09/16 21:25:00 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/09/16 21:25:00 WARN BlockManager: Block rdd_171_3 replicated to only 0 peer(s) instead of 1 peers
24/09/16 21:25:01 WARN TaskSetManager: Stage 53 contains a task of very large 

500000

In [78]:
filtered_df.cache()
filtered_df.count()

24/09/16 22:03:01 WARN TaskSetManager: Stage 11 contains a task of very large size (5038 KiB). The maximum recommended task size is 1000 KiB.
24/09/16 22:03:02 WARN TaskSetManager: Stage 12 contains a task of very large size (5038 KiB). The maximum recommended task size is 1000 KiB.


500000

In [81]:
filtered_df.unpersist(True)

DataFrame[id: bigint, name: string]

In [82]:
spark.stop()